# Trabalho Prático 3 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Esquemas pós-quânticos de assinaturas digitais* - CRYSTALS-Dilithium

Nota: Baseado no documento CRYSTALS-Dilithium (https://pq-crystals.org/dilithium/data/dilithium-specification-round3-20210208.pdf)

### Detalhes acerca do algorítmo base

Dos três esquemas pós-quânticos de assinaturas digitais propostos no enunciado do trabalho (*qTesla*, *Dilithium* e *Rainbow*), um dos que escolhemos fazer foi o **Dilithium** cuja segurança se baseia na dificuldade em encontrar vetores curtos em *lattices*. Para além disto e indo ao encontro do que é mencionado do documento mencionado acima, o 
**Dilithium** foi desenhado tendo em mente os seguintes critérios:

  - Simple to implement securely
  - Be conservative with parameters
  - Minimize the size of public key + signature
  - Be modular – easy to vary security

De seguida, tentaremos explicar unicamente as três principais funções que devem ser chamadas nesta classe: **geraChaves()**, **assina()** e **verifica()**.

**Geração da chave pública *($\rho$,$t_1$)* e da chave privada ($\rho$,K,tr,$s_1$,$s_2$,$t_0$) *(função geraChaves())***

O algorítmo de geração de chaves gera uma **matriz A**, k $\times$ l, para a qual cada uma das suas entradas é um **polinímio** pertencente ao **anel** **Rq = Zq[X]/(Xn + 1)**. Importa referir que, para este contexto em particular, , temos sempre que **q = $2^{23}$ − $2^{13}$ + 1** e **n = 256**. De seguida, o algorítmo trata de gerar 2 vetores privados, **s1** e **s2**. Finalmente, a segunda parte da chave pública é calculada como sendo **t = As1 + s2**. Todas as operações algébricas neste esquema são assumidas como sendo sobre o *polynomial ring* **Rq**.


**Assinatura *(função assina())***

  - A € Rq^k×q := ExpandA(rho)
  - μ € {0, 1}^512 := H(tr || M)
  - k := 0, (z, h) := err0
  - rho' € {0, 1}^512 := H(K || μ) (or rho' <- {0, 1}^512 for randomized signing)
  - while (z, h) = erro
  - y € ˜S_gama1^l := ExpandMask(rho', k)
    - w := Ay, ou w := NTT−1(ˆA · NTT(y))
    - w1 := HighBitsq(w, 2*gama2)
    - ˜c € {0, 1}^256 := H(μ || w1)
    - c € B_tau := SampleInBall(˜c)
    - Store c in NTT representation as ˆc = NTT(c)
    - Compute cs1 as NTT−1(ˆc · ˆs1)
    - z := y + cs1
    - Compute cs2 as NTT−1(ˆc · ˆs2)
    - r0 := LowBitsq(w − cs2, 2*gama2)
    - if ||z||inf >= gama1 − beta or ||r0||inf >= gama2 − beta, then (z, h) := erro
    - else Compute ct0 as NTT−1(ˆc ·ˆt0)
    - h := MakeHintq(−ct0,w − cs2 + ct0, 2*gama2)
    - if ||ct0||inf >= gama2 or the # of 1’s in h is greater than omega, then (z, h) := erro
    - k := k + l
    - returns (c~, z, h)
  

**Verificação *(função verifica())***

  - A € Rq^k×q := ExpandA(rho)
  - μ € {0, 1}^512 := H(H(rho || t1) || M)
  - c := SampleInBall(˜c)
  - w0 1 := UseHintq(h, Az − ct1 · 2^d, 2*gama2) -> Compute as NTT−1(ˆA · NTT(z)−NTT(c) · NTT(t1 · 2^d))
  - return [||z||inf < gama1 − beta] and [˜c = H(μ || w'1 )] and [# of 1’s in h is <= omega]
  


In [1]:
import hashlib
import numpy as np

class Dilithium(object):
    
    def __init__(self, ntt, n, q, d, tau, chall_ent, gama1, gama2, k, l, eta, beta, omega, reps, timeout=None):
        
        # Objeto da classe NTT
        self.ntt = ntt
        # Parametros:
        self.n = n
        self.q = q
        self.d = d
        self.tau = tau
        self.chall_ent = chall_ent
        self.gama1 = gama1
        self.gama2 = gama2
        self.k = k
        self.l = l
        self.eta = eta
        self.beta = beta
        self.omega = omega
        self.reps = reps
        
        # Definição dos aneis
        Zx.<x>= ZZ[]
        self.Zx = Zx
        Zq.<z>= PolynomialRing(GF(self.q))
        self.Zq = Zq
        Rq.<z>= Zq.quotient(z^self.n+1)
        self.Rq = Rq
        R.<x>= Zx.quotient(x^self.n+1)
        self.R = R
          

    '''
    Implementação da função H() responsável por gerar 
    um digest com 256-bit de comprimento (32 bytes)
    '''        
    def H(self, x):
        
        h = hashlib.shake_256()
        h.update(x)
        return h.digest(int(32))
    
    '''
    Implementação da função H() responsável por gerar 
    um digest com (256+512+256)-bit (128 bytes) de comprimento
    '''
    def H_big(self, x):
        
        h = hashlib.shake_256()
        h.update(x)
        buffer = h.digest(int(128))
        return (buffer[:32], buffer[32:96], buffer[-32:])
    
    '''
    Implementação da função SampleInBall descrita 
    no documento onde se encontra a especificação 
    do dilithium
    '''
    def SampleInBall(self, rho):
        
        # Initialize c = c0c1 . . . c255 = 00 . . . 0
        random.seed(a=rho, version=2)
        c = []
        for i in range(256):
            c.append(0)
        # for i := 256 − tau to 255
        for i in range(256 - self.tau, 256):
            # j <- {0, 1, . . . , i}
            j = random.randint(0, i)
            # s <- {0, 1}
            s = random.randint(0, 1)
            # ci := cj
            c[i] = c[j]
            # cj := (−1)^s
            c[j] = (-1)^s
        
        return c
    
    '''
    Implementação da função Power2Round
    '''
    def Power2Round(self, r, d):
        
        # r := r mod+ q
        r = r % self.q
        # r := mod ± 2^d
        r0 = r % (2^d)
        r0 = r0 - (2^(d-1))
        # return ((r − r0)/2^d, r0)
        return ((r - r0)//(2^d)), r0
    
    '''
    Implementação da função MakeHint
    '''
    def MakeHint(self, z, r, alpha):
        
        # r1 := HighBits_q(r, alpha)
        r1 = self.HighBits(r, alpha)
        # v1 := HighBits_q(r + z, alpha)
        v1 = self.HighBits(r + z, alpha)
        
        # return [r1 != v1]
        return r1 != v1
    
    
    '''
    Implementação da função UseHint
    '''
    def UseHint(self, h, r, alpha):
        
        # m := (q − 1)/alpha
        m = (self.q - 1)//alpha
        # (r1, r0) := Decompose_q(r, alpha)
        (r1, r0) = self.Decompose(r, alpha)
        # if h = 1 and r0 > 0 return (r1 + 1) mod+ m
        if h == 1 & r0 > 0:
            return (r + 1) % m
        # if h = 1 and r0 <= 0 return (r1 − 1) mod+ m
        if h == 1 & r0 <= 0:
            return (r - 1) % m
        return r1
    
    '''
    Implementação da função HighBits
    '''
    def HighBits(self, r, alpha):
        
        # (r1, r0) := Decomposeq(r, alpha)
        (r1, r0) = self.Decompose(r, alpha)
        return r1
    
    '''
    Implementação da função LowBits
    '''
    def LowBits(self, r, alpha):
        
        # (r1, r0) := Decomposeq(r, alpha)
        (r1, r0) = self.Decompose(r, alpha)
        return r0
    
    '''
    Implementação da função Decompose
    '''
    def Decompose(self, r, alpha):
        
        # r := r mod+ q
        r = r % self.q
        # r0 := r mod± alpha
        r0 = r % alpha
        r0 = r0 - (alpha//2)
        # if r − r0 = q − 1 then r1 := 0; r0 := r0 − 1
        if r - r0 == self.q - 1:
            r1 = 0
            r0 = r0 - 1
        else:
            # else r1 := (r − r0)/alpha
            r1 = (r - r0)//alpha
        
        return (r1, r0)
    
    '''
    Função que calcula o tamanho dos elementos de w atraves do seguinte:
    ||w||inf = max_i||w_i||inf, ||w|| = sqrt(||w1||^2 + . . . + ||wk||^2).
    '''
    def sizeElements(self, w):
        
        x = w % self.q
        x = x - (self.q//2)
        if x < 0:
            x = -x
        
        return x
    
    '''
    Função que conta o numero de uns de w
    '''
    def number1s(self, w):
        
        counter = 0
        aux = w
        while aux > 0:
            if aux % 2:
                counter += 1
            aux = aux // 2
        
        return counter
    
    '''
    Implementação da função ExpandA
    '''
    def ExpandA(self, x):
        
        # experiencia
        K=[]
        for i in range(self.k*self.l):
            K.append(self.Rq.random_element())
        return matrix(self.Rq, self.k, self.l, K)
    
    '''
    Implementação da função ExpandS 
    '''
    def ExpandS(self, x):
        
        # experiencia
        S=[]; S1=[]
        for i in range(self.k):
            pol=[]
            for j in range(self.n):
                pol.append(randint(1,self.eta))
                
            S.append(self.Rq(pol))
        s1=matrix(self.Rq,self.k,1,S)
        for i in range(self.l):
            pol=[]
            for j in range(self.n):
                pol.append(randint(1,self.eta))
                
            S1.append(self.Rq(pol))
        s2=matrix(self.Rq,self.l,1,S1)
        return (s1,s2)
    
    '''
    Implementação da função ExpandMask
    '''
    def ExpandMask(self, x1, x2):
        
        return 0
    
    '''
    Implementação do algorítmo de inversa de NTT a um 
    vetor cujos elementos pertencem a Rq
    '''
    def ntt_invMatrix(self, M):
        
        if type(M[0]) is list:
            res = []
            for i in range(len(M)):
                if type(M[i][0]) is list:
                    res.append([])
                    for j in range(len(M[i])):
                        res[i].append(self.ntt.ntt_inv(M[i][j]))
                else:
                    res.append(self.ntt.ntt_inv(M[i]))
                    
        else:
            res = self.ntt.ntt_inv(M)
        return res
    
    '''
    Implementação do algorítmo de NTT a uma
    matrix cujos elementos pertencem a Rq
    '''
    def nttMatrix(self, M):
        
        if type(M) is list:
            res = []
            for i in range(len(M)):
                if type(M[i]) is list:
                    res.append([])
                    for j in range(len(M[i])):
                        res[i].append(self.ntt.ntt(M[i][j]))
                else:
                    res.append(self.ntt.ntt(M[i]))
        else:
            res = self.ntt.ntt(M)
        return res
    
    '''
    Função responsável pela geração de ambas 
    as chaves, pública e privada.
    '''
    def geraChaves(self):
        
        # zeta <- {0, 1}^256
        zeta = os.urandom(32)
        # (rho, rho',K) € {0, 1}^256 × {0, 1}^512 × {0, 1}^256 := H(zeta)
        (rho, rho0, K) = self.H_big(zeta)
        # A € Rq^k×q := ExpandA(rho)
        A = self.ExpandA(rho)
        Acircum = self.nttMatrix(A)
        #(s1, s2) € Sn^l × Sn^k := ExpandS(rho')
        (s1, s2) = self.ExpandS(rho0)
        # Compute As1 as NTT^−1(ˆA · NTT(s1))
        As1 = self.ntt.ntt_inv(np.array(Acircum)*list(self.ntt.ntt(s1)))
        # t := As1 + s2
        t = As1 + s2
        # (t1, t0) := Power2Roundq(t, d)
        (t1, t0) = self.Power2Round(t,self.d)
        # tr € {0, 1}^256 := H(rho || t1)
        tr = self.H(rho + t1)
     
        pk = (rho, t1)
        sk = (rho, K, tr, s1, s2, t0)
        return pk, sk
    
    '''
    Função responsável por assinar uma mensagem 
    com uma determinada chave privada 'sk'
    '''
    def assina(self, message, sk):
        
        (rho, K, tr, s1, s2, t0) = sk
        
        # A € Rq^k×q := ExpandA(rho)
        #A = self.ExpandA(rho)
        A = matrix(self.Rq, self.k, self.l, rho)
        Acircum = self.nttMatrix(A)
        # μ € {0, 1}^512 := H(tr || M)
        niu = self.H(tr + message)
        # k := 0, (z, h) := err0
        kappa = 0
        z = 0; h = 0
        # rho' € {0, 1}^512 := H(K || μ) (or rho' <- {0, 1}^512 for randomized signing)
        rhol = self.H(K + niu)
        # ˆs1 := NTT(s1)
        s1circum = self.ntt.ntt(s1)
        # ˆs2 := NTT(s2)
        s2circum = self.ntt.ntt(s2)
        # ˆt0 := NTT(t0)
        t0circum = self.ntt.ntt(t0)
        # while (z, h) = erro
        while z == 0 and h == 0:
            
            # y € ˜S_gama1^l := ExpandMask(rho', k)
            y = self.ExpandMask(rhol, kappa)
            # w := Ay, ou w := NTT−1(ˆA · NTT(y))
            w = self.ntt.ntt_inv(Acircum * self.ntt.ntt(y))
            # w1 := HighBitsq(w, 2*gama2)
            w1 = self.HighBits(w, 2 * self.gama2)
            # ˜c € {0, 1}^256 := H(μ || w1)
            ctilde = self.H(niu, w1)
            # c € B_tau := SampleInBall(˜c)
            c = self.SampleInBall(ctilde)
            # Store c in NTT representation as ˆc = NTT(c)
            ccircum = self.NTT(c)
            # Compute cs1 as NTT−1(ˆc · ˆs1)
            cs1 = self.ntt.ntt_inv(ccircum*s1circum)
            # z := y + cs1
            z = y + cs1
            # Compute cs2 as NTT−1(ˆc · ˆs2)
            cs2 = self.ntt.ntt_inv(ccircum*s2circum)
            # r0 := LowBitsq(w − cs2, 2*gama2)
            r0 = self.LowBits(w-cs2, 2 * self.gama2)
            # if ||z||inf >= gama1 − beta or ||r0||inf >= gama2 − beta, then (z, h) := erro
            if self.sizeElements(z) >= (self.gama1 - self.beta) or self.sizeElements(r0) >= (self.gama2 - self.beta):
                z = 0
                h = 0
            else:
                # Compute ct0 as NTT−1(ˆc ·ˆt0)
                ct0 = self.ntt.ntt_inv(ccircum * t0circum)
                # h := MakeHintq(−ct0,w − cs2 + ct0, 2*gama2)
                h = self.MakeHint(-ct0, w - cs2 + ct0, 2 * self.gama2)
                # if ||ct0||inf >= gama2 or the # of 1’s in h is greater than omega, then (z, h) := erro
                if self.sizeElements(ct0) >= self.gama2 or self.number1s(h) > self.omega:
                    z = 0
                    h = 0
            # k := k + l
            kappa = kappa + l
    
        return (ctilde, z, h)
    
    '''
    Função que verifica a assinatura de uma mensagem 
    com uma determinada chave pública
    '''
    def verifica(self, message, pk, sigma):
        
        (rho, t1) = pk
        (ctilde, z, h) = sigma
        # A € Rq^k×q := ExpandA(rho)
        A = self.ExpandA(rho)
        Acircum = self.nttMatrix(A)
        # μ € {0, 1}^512 := H(H(rho || t1) || M)
        niu = self.H(self.H(rho + t1) + message)
        # c := SampleInBall(˜c)
        c = self.SampleInBall(ctilde)
        # w0 1 := UseHintq(h, Az − ct1 · 2^d, 2*gama2) -> Compute as NTT−1(ˆA · NTT(z)−NTT(c) · NTT(t1 · 2^d))
        wl1 = self.UseHint(h, self.ntt.ntt_inv((Acircum * self.ntt.ntt(z)) - (self.ntt.ntt(c) * self.ntt.ntt(t1*(2^13)))))
        # return [||z||inf < gama1 − beta] and [˜c = H(μ || w'1 )] and [# of 1’s in h is <= omega]
        return self.sizeElements(z) < self.gama1 - self.beta and ctilde == self.H(niu + wl1) and self.number1s(h) <= self.omega
    
    
    

In [2]:
# Classe que implementa o NTT (Number Theoretic Transform)

class NTT(object):
   
    def __init__(self, n=128, q=None):
        if not  n in [32,64,128,256,512,1024,2048]:
            raise ValueError("improper argument ",n)
        self.n = n  
        if not q:
            self.q = 1 + 2*n
            while True:
                if (self.q).is_prime():
                    break
                self.q += 2*n
        else:
            if q % (2*n) != 1:
                raise ValueError("Valor de 'q' não verifica a condição NTT")
            self.q = q
             
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen()
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def ntt_inv(self,ff):                 
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)


#### Testagem da classe definida acima:

In [ ]:
import string

# Parametros iniciais
n = 256
# prime 8380417 = 2^23 − 2^13 + 1.
q = 8380417
# [dropped bits from t]
d = 13
# [# of ±1’s in c]
tau = 39
# challenge entropy
chall_ent = 192
# [y coefficient range]
gama1 = 2^17
# [low-order rounding range]
gama2 = 95232
# [dimensions of A]
k = 4
l = 4
# [secret key range]
eta = 2
# [tau · eta]
beta = 78
# [max. # of 1’s in the hint h] 
omega = 80
# Repetitions
reps = 4.25

# Teste
print("[Testagem da classe acima]:\n")

# Inicialização do objeto NTT

ntt = NTT(n=n, q=q)

dilithium = Dilithium(ntt, n, q, d, tau, chall_ent, gama1, gama2, k, l, eta, beta, omega, reps)

# Inserir uam mensagem a assinar
message = input("Insira uma mensagem a assinar: ")

# Gerar as Chaves
(pk, sk) = dilithium.geraChaves()
# Assinar a mensagem com a chave privada
s = dilithium.assina(message, sk)
#print(s)
# Verificar a assinatura com a chave publica
res = dilithium.verifica(message, s, pk)

if res == 0:
    print("A assinatura digital é válida!!")
else:
    print("A assinatura digital não é válida!!")